### Importação de configurações e funções

In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *
from Utils.utils import *

apply_storage_config(spark)

In [0]:
from pyspark.sql.types import IntegerType, DoubleType, StringType
from pyspark.sql.functions import (
    col,
    lpad,
    current_timestamp,
    max as spark_max,
)
import re, os

### Definição dos paths

In [0]:
path_storage_bronze = f"{bronze_path}"
path_storage_silver = f"{silver_path}"

print(f"Bronze path: {path_storage_bronze}")
print(f"Silver path: {path_storage_silver}")

### Criação da tabela de controle de transformações Silver

In [0]:
def criar_tabela_controle_silver():
    """
    Cria tabela para rastrear transformações realizadas na Silver
    """
    spark.sql("""
        CREATE TABLE IF NOT EXISTS silver.transformacoes_processadas (
            tabela_silver STRING,
            data_ultima_transformacao TIMESTAMP,
            total_registros BIGINT,
            data_processamento TIMESTAMP
        )
        USING DELTA
    """)
    print("Tabela de controle de transformações Silver criada!")

criar_tabela_controle_silver()

### Funções auxiliares para transformações

In [0]:
def obter_data_ultima_transformacao(tabela_silver):
    """
    Obtém a data da última transformação realizada para uma tabela
    """
    try:
        resultado = spark.sql(f"""
            SELECT MAX(data_ultima_transformacao) as ultima_data
            FROM silver.transformacoes_processadas
            WHERE tabela_silver = '{tabela_silver}'
        """).collect()
        
        if resultado and resultado[0].ultima_data:
            return resultado[0].ultima_data
        else:
            return None
    except:
        return None

In [0]:
def registrar_transformacao_silver(tabela_silver, total_registros):
    """
    Registra transformação realizada na Silver
    """
    spark.sql(f"""
        INSERT INTO silver.transformacoes_processadas
        VALUES (
            '{tabela_silver}',
            current_timestamp(),
            {total_registros},
            current_timestamp()
        )
    """)

### Limpeza e tratamentos

#### Tratamentos NCM

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES NCM")
print("=" * 60)

# Obter data da última transformação
ultima_transformacao_ncm = obter_data_ultima_transformacao("ncm_tratado")

# Ler NCM da Bronze
df_ncm = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/ncm/")

# Filtro incremental: apenas registros novos desde a última transformação
if ultima_transformacao_ncm:
    df_ncm = df_ncm.filter(col("ingestion_dt") > ultima_transformacao_ncm)
    print(f"Filtrando NCM desde: {ultima_transformacao_ncm}")
else:
    print("Primeira execução: processando todos os NCM")

print(f"Total de registros NCM para processar: {df_ncm.count()}")

In [0]:
# Sequência de tratamentos NCM
# 1. strip/trim
df_ncm = df_ncm.withColumn("NO_NCM_POR", col("NO_NCM_POR").cast("string"))
for col_name in ["NO_NCM_POR"]:
    if col_name in df_ncm.columns:
        df_ncm = df_ncm.withColumn(col_name, col(col_name).cast("string"))

# 2. Nulificar colunas em branco
for col_name in ["NO_NCM_POR"]:
    if col_name in df_ncm.columns:
        df_ncm = df_ncm.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None).otherwise(col(col_name))
        )

# 3. LPAD dos NCM
df_ncm = df_ncm.withColumn("CO_NCM", lpad(col("CO_NCM"), 8, "0"))
df_ncm = df_ncm.withColumn("CO_SH6", lpad(col("CO_SH6"), 6, "0"))

# 4. Remover colunas desnecessárias
df_ncm_filtrado = df_ncm.drop("NO_NCM_ESP", "NO_NCM_ING", "ingestion_dt", "tipo_operacao", "origin_path_name")

print(f"NCM transformados: {df_ncm_filtrado.count()} registros")

In [0]:
# Escrita incremental (append) na Silver
(
    df_ncm_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/ncm_tratado")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.ncm_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/ncm_tratado'
""".format(path_storage_silver=path_storage_silver))

# Registrar transformação
registrar_transformacao_silver("ncm_tratado", df_ncm_filtrado.count())

print("NCM salvos em Silver")

In [0]:
%sql
SELECT * FROM silver.transformacoes_processadas

#### Tratamentos NCM_SH

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES NCM_SH")
print("=" * 60)

ultima_transformacao_ncm_sh = obter_data_ultima_transformacao("ncm_sh_tratado")

df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")

if ultima_transformacao_ncm_sh:
    df_ncm_sh = df_ncm_sh.filter(col("ingestion_dt") > ultima_transformacao_ncm_sh)
    print(f"Filtrando NCM_SH desde: {ultima_transformacao_ncm_sh}")
else:
    print("Primeira execução: processando todos os NCM_SH")

print(f"Total de registros NCM_SH para processar: {df_ncm_sh.count()}")

In [0]:
# Sequência de tratamentos NCM_SH
for col_name in ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"]:
    if col_name in df_ncm_sh.columns:
        df_ncm_sh = df_ncm_sh.withColumn(col_name, col(col_name).cast("string"))

# Nulificar colunas em branco
for col_name in ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"]:
    if col_name in df_ncm_sh.columns:
        df_ncm_sh = df_ncm_sh.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None)
        )

# LPAD
df_ncm_sh = df_ncm_sh.withColumn("CO_SH6", lpad(col("CO_SH6"), 6, "0"))
df_ncm_sh = df_ncm_sh.withColumn("CO_SH4", lpad(col("CO_SH4"), 4, "0"))
df_ncm_sh = df_ncm_sh.withColumn("CO_SH2", lpad(col("CO_SH2"), 2, "0"))

# Remover colunas desnecessárias
df_ncm_sh_filtrado = df_ncm_sh.drop(
    "NO_SH6_ESP", "NO_SH6_ING", "NO_SH4_ESP", "NO_SH4_ING",
    "NO_SH2_ESP", "NO_SH2_ING", "NO_SEC_ESP", "NO_SEC_ING",
    "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f"NCM_SH transformados: {df_ncm_sh_filtrado.count()} registros")

In [0]:
(
    df_ncm_sh_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/ncm_sh_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.ncm_sh_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/ncm_sh_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("ncm_sh_tratado", df_ncm_sh_filtrado.count())

print("NCM_SH salvos em Silver")

#### Tratamentos NCM_ISIC

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES NCM_ISIC")
print("=" * 60)

ultima_transformacao_ncm_isic = obter_data_ultima_transformacao("ncm_isic_tratado")

df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic/")

if ultima_transformacao_ncm_isic:
    df_ncm_isic = df_ncm_isic.filter(col("ingestion_dt") > ultima_transformacao_ncm_isic)
    print(f"Filtrando NCM_ISIC desde: {ultima_transformacao_ncm_isic}")
else:
    print("Primeira execução: processando todos os NCM_ISIC")

print(f"Total de registros NCM_ISIC para processar: {df_ncm_isic.count()}")

In [0]:
# Sequência de tratamentos NCM_ISIC
for col_name in ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"]:
    if col_name in df_ncm_isic.columns:
        df_ncm_isic = df_ncm_isic.withColumn(col_name, col(col_name).cast("string"))

# Nulificar colunas em branco
for col_name in ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"]:
    if col_name in df_ncm_isic.columns:
        df_ncm_isic = df_ncm_isic.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None)
        )

# LPAD
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_CLASSE", lpad(col("CO_ISIC_CLASSE"), 4, "0"))
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_GRUPO", lpad(col("CO_ISIC_GRUPO"), 3, "0"))
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_DIVISAO", lpad(col("CO_ISIC_DIVISAO"), 2, "0"))

# Remover colunas desnecessárias
df_ncm_isic_filtrado = df_ncm_isic.drop(
    "NO_ISIC_CLASSE_ESP", "NO_ISIC_CLASSE_ING", "NO_ISIC_GRUPO_ESP", "NO_ISIC_GRUPO_ING",
    "NO_ISIC_DIVISAO_ESP", "NO_ISIC_DIVISAO_ING", "NO_ISIC_SECAO_ESP", "NO_ISIC_SECAO_ING",
    "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f"NCM_ISIC transformados: {df_ncm_isic_filtrado.count()} registros")

In [0]:
(
    df_ncm_isic_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/ncm_isic_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.ncm_isic_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/ncm_isic_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("ncm_isic_tratado", df_ncm_isic_filtrado.count())

print("NCM_ISIC salvos em Silver")

#### Tratamentos NCM_CGCE

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES NCM_CGCE")
print("=" * 60)

ultima_transformacao_ncm_cgce = obter_data_ultima_transformacao("ncm_cgce_tratado")

df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce/")

if ultima_transformacao_ncm_cgce:
    df_ncm_cgce = df_ncm_cgce.filter(col("ingestion_dt") > ultima_transformacao_ncm_cgce)
    print(f"Filtrando NCM_CGCE desde: {ultima_transformacao_ncm_cgce}")
else:
    print("Primeira execução: processando todos os NCM_CGCE")

print(f"Total de registros NCM_CGCE para processar: {df_ncm_cgce.count()}")

In [0]:
# Sequência de tratamentos NCM_CGCE
for col_name in ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"]:
    if col_name in df_ncm_cgce.columns:
        df_ncm_cgce = df_ncm_cgce.withColumn(col_name, col(col_name).cast("string"))

# Nulificar colunas em branco
for col_name in ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"]:
    if col_name in df_ncm_cgce.columns:
        df_ncm_cgce = df_ncm_cgce.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None).otherwise(col(col_name))
        )

# LPAD
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N3", lpad(col("CO_CGCE_N3"), 3, "0"))
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N2", lpad(col("CO_CGCE_N2"), 2, "0"))
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N1", lpad(col("CO_CGCE_N1"), 1, "0"))

# Remover colunas desnecessárias
df_ncm_cgce_filtrado = df_ncm_cgce.drop(
    "NO_CGCE_N3_ESP", "NO_CGCE_N3_ING", "NO_CGCE_N2_ESP", "NO_CGCE_N2_ING",
    "NO_CGCE_N1_ESP", "NO_CGCE_N1_ING",
    "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f"✔ NCM_CGCE transformados: {df_ncm_cgce_filtrado.count()} registros")

In [0]:
(
    df_ncm_cgce_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/ncm_cgce_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.ncm_cgce_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/ncm_cgce_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("ncm_cgce_tratado", df_ncm_cgce_filtrado.count())

print("NCM_CGCE salvos em Silver")

#### Tratamentos NCM_FAT_AGREG

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES NCM_FAT_AGREG")
print("=" * 60)

ultima_transformacao_ncm_fat = obter_data_ultima_transformacao("ncm_fat_agreg_tratado")

df_ncm_fat_agreg = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_fat_agreg/")

if ultima_transformacao_ncm_fat:
    df_ncm_fat_agreg = df_ncm_fat_agreg.filter(col("ingestion_dt") > ultima_transformacao_ncm_fat)
    print(f"Filtrando NCM_FAT_AGREG desde: {ultima_transformacao_ncm_fat}")
else:
    print("Primeira execução: processando todos os NCM_FAT_AGREG")

print(f"Total de registros NCM_FAT_AGREG para processar: {df_ncm_fat_agreg.count()}")

In [0]:
# Sequência de tratamentos NCM_FAT_AGREG
for col_name in ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"]:
    if col_name in df_ncm_fat_agreg.columns:
        df_ncm_fat_agreg = df_ncm_fat_agreg.withColumn(col_name, col(col_name).cast("string"))

# Nulificar colunas em branco
for col_name in ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"]:
    if col_name in df_ncm_fat_agreg.columns:
        df_ncm_fat_agreg = df_ncm_fat_agreg.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None)
        )

# LPAD
df_ncm_fat_agreg = df_ncm_fat_agreg.withColumn("CO_FAT_AGREG", lpad(col("CO_FAT_AGREG"), 2, "0"))

# Remover colunas desnecessárias
df_ncm_fat_agreg_filtrado = df_ncm_fat_agreg.drop(
    "NO_FAT_AGREG_GP", "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f"NCM_FAT_AGREG transformados: {df_ncm_fat_agreg_filtrado.count()} registros")

In [0]:
(
    df_ncm_fat_agreg_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/ncm_fat_agreg_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.ncm_fat_agreg_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/ncm_fat_agreg_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("ncm_fat_agreg_tratado", df_ncm_fat_agreg_filtrado.count())

print("NCM_FAT_AGREG salvos em Silver")

#### Tratamentos PAIS

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES PAIS")
print("=" * 60)

ultima_transformacao_pais = obter_data_ultima_transformacao("pais_tratado")

df_pais = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais")

if ultima_transformacao_pais:
    df_pais = df_pais.filter(col("ingestion_dt") > ultima_transformacao_pais)
    print(f"Filtrando PAIS desde: {ultima_transformacao_pais}")
else:
    print("Primeira execução: processando todos os PAIS")

print(f"Total de registros PAIS para processar: {df_pais.count()}")

In [0]:
# Sequência de tratamentos PAIS
df_pais = df_pais.withColumn("NO_PAIS", col("NO_PAIS").cast("string"))

# Nulificar colunas em branco
df_pais = df_pais.withColumn(
    "NO_PAIS",
    when(col("NO_PAIS").rlike("^\\s*$"), None).otherwise(col("NO_PAIS"))
    
)

# CAST INT
df_pais = df_pais.withColumn("CO_PAIS", col("CO_PAIS").cast("int"))
df_pais = df_pais.withColumn("CO_PAIS_ISON3", col("CO_PAIS_ISON3").cast("int"))

# Remover colunas desnecessárias
df_pais_filtrado = df_pais.drop(
    "NO_PAIS_ESP", "NO_PAIS_ING", "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f"PAIS transformados: {df_pais_filtrado.count()} registros")

In [0]:
(
    df_pais_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/pais_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.pais_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/pais_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("pais_tratado", df_pais_filtrado.count())

print("PAIS salvos em Silver")

#### Tratamentos PAIS_BLOCO

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES PAIS_BLOCO")
print("=" * 60)

ultima_transformacao_pais_bloco = obter_data_ultima_transformacao("pais_bloco_tratado")

df_pais_bloco = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais_bloco")

if ultima_transformacao_pais_bloco:
    df_pais_bloco = df_pais_bloco.filter(col("ingestion_dt") > ultima_transformacao_pais_bloco)
    print(f"Filtrando PAIS_BLOCO desde: {ultima_transformacao_pais_bloco}")
else:
    print("Primeira execução: processando todos os PAIS_BLOCO")

print(f"Total de registros PAIS_BLOCO para processar: {df_pais_bloco.count()}")

In [0]:
# Sequência de tratamentos PAIS_BLOCO
df_pais_bloco = df_pais_bloco.withColumn("NO_BLOCO", col("NO_BLOCO").cast("string"))

# CAST INT
df_pais_bloco = df_pais_bloco.withColumn("CO_PAIS", col("CO_PAIS").cast("int"))
df_pais_bloco = df_pais_bloco.withColumn("CO_BLOCO", col("CO_BLOCO").cast("int"))

# Remover colunas desnecessárias
df_pais_bloco_filtrado = df_pais_bloco.drop(
    "NO_BLOCO_ESP", "NO_BLOCO_ING", "ingestion_dt", "tipo_operacao", "origin_path_name"
)

print(f" PAIS_BLOCO transformados: {df_pais_bloco_filtrado.count()} registros")

In [0]:
(
    df_pais_bloco_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/pais_bloco_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.pais_bloco_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/pais_bloco_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("pais_bloco_tratado", df_pais_bloco_filtrado.count())

print("PAIS_BLOCO salvos em Silver")

#### Tratamentos UF

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES UF")
print("=" * 60)

ultima_transformacao_uf = obter_data_ultima_transformacao("uf_tratado")

df_uf = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf")

if ultima_transformacao_uf:
    df_uf = df_uf.filter(col("ingestion_dt") > ultima_transformacao_uf)
    print(f"Filtrando UF desde: {ultima_transformacao_uf}")
else:
    print("Primeira execução: processando todos os UF")

print(f"Total de registros UF para processar: {df_uf.count()}")

In [0]:
# Sequência de tratamentos UF
for col_name in ["SG_UF", "NO_UF", "NO_REGIAO"]:
    if col_name in df_uf.columns:
        df_uf = df_uf.withColumn(col_name, col(col_name).cast("string"))


# CAST INT
df_uf = df_uf.withColumn("CO_UF", col("CO_UF").cast("int"))

# Remover colunas desnecessárias
df_uf_filtrado = df_uf.drop("ingestion_dt", "tipo_operacao", "origin_path_name")

print(f"UF transformados: {df_uf_filtrado.count()} registros")

In [0]:
(
    df_uf_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/uf_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.uf_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/uf_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("uf_tratado", df_uf_filtrado.count())

print("UF salvos em Silver")

#### Tratamentos UF_MUN

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES UF_MUN")
print("=" * 60)

ultima_transformacao_uf_mun = obter_data_ultima_transformacao("uf_mun_tratado")

df_uf_mun = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf_mun")

if ultima_transformacao_uf_mun:
    df_uf_mun = df_uf_mun.filter(col("ingestion_dt") > ultima_transformacao_uf_mun)
    print(f"Filtrando UF_MUN desde: {ultima_transformacao_uf_mun}")
else:
    print("Primeira execução: processando todos os UF_MUN")

print(f"Total de registros UF_MUN para processar: {df_uf_mun.count()}")

In [0]:
# Sequência de tratamentos UF_MUN
for col_name in ["NO_MUN", "SG_UF"]:
    if col_name in df_uf_mun.columns:
        df_uf_mun = df_uf_mun.withColumn(col_name, col(col_name).cast("string"))

# Nulificar colunas em branco
for col_name in ["NO_MUN", "SG_UF"]:
    if col_name in df_uf_mun.columns:
        df_uf_mun = df_uf_mun.withColumn(
            col_name,
            when(col(col_name).rlike("^\\s*$"), None).otherwise(col(col_name))
        )

# CAST INT
df_uf_mun = df_uf_mun.withColumn("CO_MUN_GEO", col("CO_MUN_GEO").cast("int"))

# Remover colunas desnecessárias
df_uf_mun_filtrado = df_uf_mun.drop(
    "ingestion_dt", "tipo_operacao", "origin_path_name", "NO_MUN_MIN"
)

print(f"UF_MUN transformados: {df_uf_mun_filtrado.count()} registros")

In [0]:
(
    df_uf_mun_filtrado
    .write
    .format("delta")
    .mode("append")
    .save(f"{path_storage_silver}/balancacomercial/uf_mun_tratado")
)

spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.uf_mun_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/uf_mun_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("uf_mun_tratado", df_uf_mun_filtrado.count())

print("UF_MUN salvos em Silver")

#### Tratamentos EXP

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES EXP (EXPORTAÇÃO)")
print("=" * 60)

ultima_transformacao_exp = obter_data_ultima_transformacao("exp_tratado")

df_exp = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/exp/")

if ultima_transformacao_exp:
    df_exp = df_exp.filter(col("ingestion_dt") > ultima_transformacao_exp)
    print(f" Filtrando EXP desde: {ultima_transformacao_exp}")
else:
    print(" Primeira execução: processando todos os EXP")

print(f"Total de registros EXP para processar: {df_exp.count()}")

In [0]:
# 1. Remover colunas desnecessárias
df_exp_filtrado = df_exp.drop(
    "ingestion_dt",
    "origin_path_name"
)

# 2. Transformar tipos de dados
df_exp_filtrado = df_exp_filtrado.select(
    col("CO_ANO").cast(IntegerType()).alias("CO_ANO"),
    col("CO_MES").cast(IntegerType()).alias("CO_MES"),
    
    # Colunas de identificação (manter como STRING)
    col("CO_NCM").alias("CO_NCM"),
    col("CO_UNID").alias("CO_UNID"),
    col("CO_PAIS").alias("CO_PAIS"),
    col("SG_UF_NCM").alias("SG_UF_NCM"),
    col("CO_VIA").alias("CO_VIA"),
    col("CO_URF").alias("CO_URF"),
    
    # Colunas numéricas
    col("QT_ESTAT").cast(IntegerType()).alias("QT_ESTAT"),
    col("KG_LIQUIDO").cast(IntegerType()).alias("KG_LIQUIDO"),
    col("VL_FOB").cast(DoubleType()).alias("VL_FOB"),
    
    # Metadados
    col("tipo_operacao").alias("TIPO_OPERACAO"),
    current_timestamp().alias("DATA_TRANSFORMACAO")
)

print(f"EXP transformados: {df_exp_filtrado.count()} registros")


In [0]:
# Escrita incremental (append) na Silver
(
    df_exp_filtrado
    .write
    .format("delta")
    .mode("append")
    .partitionBy("CO_ANO", "CO_MES")
    .save(f"{path_storage_silver}/balancacomercial/exp_tratado")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.exp_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/exp_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("exp_tratado", df_exp_filtrado.count())

print("EXP salvos em Silver")

#### Tratamentos importação

In [0]:
print("=" * 60)
print("INICIANDO TRANSFORMAÇÕES IMP (IMPORTAÇÃO)")
print("=" * 60)

ultima_transformacao_imp = obter_data_ultima_transformacao("imp_tratado")

df_imp = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/imp/")

if ultima_transformacao_imp:
    df_imp = df_imp.filter(col("ingestion_dt") > ultima_transformacao_imp)
    print(f"Filtrando IMP desde: {ultima_transformacao_imp}")
else:
    print("Primeira execução: processando todos os IMP")

print(f"Total de registros IMP para processar: {df_imp.count()}")

In [0]:
# Sequência de tratamentos IMP
# 1. Remover colunas desnecessárias
df_imp_filtrado = df_imp.drop(
    "ingestion_dt",
    "origin_path_name"
)

# 2. Transformar tipos de dados
df_imp_filtrado = df_imp_filtrado.select(
    col("CO_ANO").cast(IntegerType()).alias("CO_ANO"),
    col("CO_MES").cast(IntegerType()).alias("CO_MES"),
    
    # Colunas de identificação (manter como STRING)
    col("CO_NCM").alias("CO_NCM"),
    col("CO_UNID").alias("CO_UNID"),
    col("CO_PAIS").alias("CO_PAIS"),
    col("SG_UF_NCM").alias("SG_UF_NCM"),
    col("CO_VIA").alias("CO_VIA"),
    col("CO_URF").alias("CO_URF"),
    
    # Colunas numéricas
    col("QT_ESTAT").cast(IntegerType()).alias("QT_ESTAT"),
    col("KG_LIQUIDO").cast(IntegerType()).alias("KG_LIQUIDO"),
    col("VL_FOB").cast(DoubleType()).alias("VL_FOB"),
    col("VL_FRETE").cast(DoubleType()).alias("VL_FRETE"),
    col("VL_SEGURO").cast(DoubleType()).alias("VL_SEGURO"),
    
    # Metadados
    col("tipo_operacao").alias("TIPO_OPERACAO"),
    current_timestamp().alias("DATA_TRANSFORMACAO")
)

print(f"IMP transformados: {df_imp_filtrado.count()} registros")

In [0]:
# Escrita incremental (append) na Silver
(
    df_imp_filtrado
    .write
    .format("delta")
    .mode("append")
    .partitionBy("CO_ANO", "CO_MES")
    .save(f"{path_storage_silver}/balancacomercial/imp_tratado")
)

# Criar tabela se não existir
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver.imp_tratado
    USING DELTA
    LOCATION '{path_storage_silver}/balancacomercial/imp_tratado'
""".format(path_storage_silver=path_storage_silver))

registrar_transformacao_silver("imp_tratado", df_imp_filtrado.count())

print("IMP salvos em Silver")

### Resumo das transformações

In [0]:
print("=" * 60)
print("RESUMO DAS TRANSFORMAÇÕES SILVER")
print("=" * 60)

df_resumo = spark.sql("""
    SELECT
        tabela_silver,
        COUNT(*) as total_transformacoes,
        SUM(total_registros) as total_registros_processados,
        MAX(data_ultima_transformacao) as ultima_transformacao
    FROM silver.transformacoes_processadas
    GROUP BY tabela_silver
    ORDER BY tabela_silver
""")

df_resumo.display()

print("\n Transformações Silver finalizadas com sucesso!")